In [2]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

In [3]:
f = pd.read_csv('data2.csv')

In [32]:
x = 1
y = 2

SyntaxError: invalid syntax (<ipython-input-32-516d9a6bf5ae>, line 3)

In [4]:
f.head(n=6)

,职业,ID,祝福,唱歌,一觉,三觉,其他,三攻,唱歌后三攻,力智总和,相对提升伤害,双C提升伤害(迫害奶萝),Unnamed: 12,Unnamed: 13
0,奶萝,Deer °,44357,11089,39924,11578,599,3206,4608,107547,38206,35320,图片,NaN
1,奶妈,蝶舞花翩翩,43174,6476,39650,11498,991,3740,4301,101789,34983,34983,NaN,NaN
2,奶萝,凌采露华,35779,8944,41626,12071,777,4157,5196,99197,38422,35600,NaN,NaN
3,奶萝,伊莉雅,43892,10973,38422,11142,599,3223,4028,105028,34586,32042,NaN,NaN
4,奶妈,桃花舞翩翩,39212,5881,38029,11028,991,3470,3990,95141,31634,31634,NaN,NaN
5,奶萝,惊星落鸿,38365,9591,37342,11202,742,3446,4307,97242,33683,31236,NaN,NaN


In [5]:
carry_str = 4800
job_base = 950
carry_atk = 3000
carry_str_br = 2.31*(carry_str-job_base)+4581 + carry_str
carry_str_br

18274.5

In [6]:
def calNormalBuffStr(row):
    if row[u'祝福'] == '/':
        row[u'祝福'] =  0
    if row[u'唱歌'] == '/':
        row[u'唱歌'] =  0
    if row[u'其他'] == '/':
        row[u'其他'] = 0
    return int(row[u'祝福'])+int(row[u'唱歌'])+int(row[u'其他'])

def calNormalBuff(row, carry_number):
    if row[u'职业'] == '奶萝':
        atk = row[u'唱歌后三攻']*(1+(carry_number-1)*0.85)/carry_number
        str_int = row[u'常驻力智']*(1+(carry_number-1)*0.85)/carry_number
    else:
        atk = row[u'唱歌后三攻']
        str_int = row[u'常驻力智']
    return ((str_int+carry_str_br)/250) * (carry_atk+ atk)/100

def calTotallBuff(row, carry_number):
    atk_main = row[u'唱歌后三攻']
    str_main = row[u'力智总和']
    if row[u'职业'] == '奶萝':
        atk_oth = row[u'唱歌后三攻']*0.85
        str_oth = row[u'常驻力智']*0.85+row[u'一觉']+row[u'三觉']
    else:
        atk_oth = row[u'唱歌后三攻']
        str_oth = row[u'力智总和']
    main_buff = ((str_main+carry_str_br)/250) * (carry_atk+ atk_main)/100
    oth_buff = ((str_oth+carry_str_br)/250) * (carry_atk+ atk_oth)/100 * (carry_number-1)
    return (main_buff+oth_buff)/carry_number

def normalization(row, row_name, min_value, max_value):
    return (row[row_name]-min_value)/(max_value-min_value)

def normalization_zerobase(row,row_name, max_value):
    return (row[row_name])/(max_value)

In [7]:
f[u'常驻力智'] = f.apply(lambda x : calNormalBuffStr(x), axis=1)
f[u'常驻提升'] = f.apply(lambda x: calNormalBuff(x,1), axis=1)
f[u'常驻提升-2C'] = f.apply(lambda x: calNormalBuff(x,2), axis=1)

f[u'相对提升伤害-2C'] = f.apply(lambda x: calTotallBuff(x,2), axis=1)

f[u'力智总和-归一化'] = f.apply(lambda x: normalization(x,u'力智总和', f[u'力智总和'].min(),f[u'力智总和'].max()), axis=1)

f[u'唱歌后三攻-归一化'] = f.apply(lambda x: normalization(x,u'唱歌后三攻',f[u'唱歌后三攻'].min(), f[u'唱歌后三攻'].max()), axis=1)

f[u'常驻提升-归一化'] = f.apply(lambda x: normalization(x,u'常驻提升',f[u'常驻提升'].min(), f[u'常驻提升'].max()), axis=1)

f[u'相对提升伤害-归一化'] = f.apply(lambda x: normalization(x,u'相对提升伤害',f[u'相对提升伤害'].min(), f[u'相对提升伤害'].max()), axis=1)
f[u'常驻提升-2C-归一化'] = f.apply(lambda x: normalization(x, u'常驻提升-2C', f[u'常驻提升-2C'].min(),f[u'常驻提升-2C'].max()), axis=1)
f[u'相对提升伤害2C-归一化'] = f.apply(lambda x: normalization(x,u'相对提升伤害-2C', f[u'相对提升伤害-2C'].min(), f[u'相对提升伤害-2C'].max()), axis=1)

In [8]:

f[u'力智总和-归一化'] = f.apply(lambda x: normalization_zerobase(x,u'力智总和', f[u'力智总和'].max()), axis=1)

f[u'唱歌后三攻-归一化'] = f.apply(lambda x: normalization_zerobase(x,u'唱歌后三攻', f[u'唱歌后三攻'].max()), axis=1)

f[u'常驻提升-归一化'] = f.apply(lambda x: normalization_zerobase(x, u'常驻提升', f[u'常驻提升'].max()), axis=1)

f[u'相对提升伤害-归一化'] = f.apply(lambda x: normalization_zerobase(x,u'相对提升伤害', f[u'相对提升伤害'].max()), axis=1)
f[u'常驻提升-2C-归一化'] = f.apply(lambda x: normalization_zerobase(x,u'常驻提升-2C', f[u'常驻提升-2C'].max()), axis=1)
f[u'相对提升伤害2C-归一化'] = f.apply(lambda x: normalization_zerobase(x,u'相对提升伤害-2C', f[u'相对提升伤害-2C'].max()), axis=1)

In [15]:
f[['ID',u'职业',u'力智总和-归一化', u'常驻提升-归一化',u'唱歌后三攻-归一化',u'相对提升伤害-归一化', u'相对提升伤害2C-归一化']]

,ID,职业,力智总和-归一化,常驻提升-归一化,唱歌后三攻-归一化,相对提升伤害-归一化,相对提升伤害2C-归一化
0,Deer °,奶萝,1.000000,1.000000,0.886836,0.994378,0.992131
1,蝶舞花翩翩,奶妈,0.946461,0.889869,0.827752,0.910494,0.983038
2,凌采露华,奶萝,0.922360,0.924434,1.000000,1.000000,1.000000
3,伊莉雅,奶萝,0.976578,0.916543,0.775212,0.900161,0.900068
4,桃花舞翩翩,奶妈,0.884646,0.795628,0.767898,0.823330,0.889051
5,惊星落鸿,奶萝,0.904181,0.865491,0.828907,0.876659,0.877457
6,RY丶god,奶爸,0.888867,0.765590,0.735758,0.806881,0.871284
7,羽生結弦,奶妈,0.888876,0.795866,0.732679,0.805008,0.869249
8,マネキン嬢,奶萝,0.926190,0.870613,0.776174,0.861121,0.861811
9,ジュリア,奶妈,0.792175,0.672889,0.852002,0.797902,0.861801


In [10]:
rows=f.head(n=5)[['ID',u'职业',u'唱歌后三攻-归一化', u'力智总和-归一化', u'常驻提升-归一化',u'常驻提升-2C-归一化', u'相对提升伤害-归一化', u'相对提升伤害2C-归一化']]

In [11]:
for r in rows.iterrows():
    data_tuple = r[1]
    result_text = """ {{
    value: [ {b} , {c},{d}, {e},{f}, {g} ] ,
    name: '{a}' }},
    """.format(a=data_tuple[0], b=data_tuple[2]*100, c=data_tuple[3]*100,
               d=data_tuple[4]*100, e=data_tuple[5]*100, f=data_tuple[6]*100,
              g=data_tuple[7]*100)
    print(  result_text)

 {
    value: [ 88.68360277136259 , 100.0,100.0, 100.0,99.43782208109937, 99.21314369184995 ] ,
    name: 'Deer °' },
    
 {
    value: [ 82.7752117013087 , 94.64606172185184,88.98687931477592, 98.8100656544247,91.04939878194784, 98.30381706975506 ] ,
    name: '蝶舞花翩翩' },
    
 {
    value: [ 100.0 , 92.23595265325858,92.44336144122222, 92.53601360542505,100.0, 100.0 ] ,
    name: '凌采露华' },
    
 {
    value: [ 77.5211701308699 , 97.65776823156388,91.6542838965611, 91.90277903709445,90.01613658841289, 90.00681846664203 ] ,
    name: '伊莉雅' },
    
 {
    value: [ 76.78983833718245 , 88.4645782774043,79.56275374951481, 88.34561883923709,82.33303836343762, 88.90508901633135 ] ,
    name: '桃花舞翩翩' },
    


In [13]:
for r in rows.iterrows():
    data_tuple = r[1]
    print("'"+data_tuple[0]+"',")

'Deer °',
'蝶舞花翩翩',
'凌采露华',
'伊莉雅',
'桃花舞翩翩',
